> __Purpose__: The purpose of this notebook is to take the best meta model from NB75, then test it on unseen data, and use this to determine the performance of how the validation checks are performing (e.g. are the validation checks working as expected).

In [1]:
#no_test

#data structure imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from os import listdir

#python imports
import random

#modeling imports
from spacy.util import fix_random_seed
from ssda_nlp.collate import *
from ssda_nlp.split_data import *
from ssda_nlp.modeling import *
from ssda_nlp.preprocessing import *
from ssda_nlp.model_performance_utils import *
from ssda_nlp.utility import *
from ssda_nlp.relationships import *
from ssda_nlp.full_volume import *

#if you want the same model everytime:
seed = 2436
random.seed(seed)
fix_random_seed(seed)

## From NB 74
- Update to best meta model. Doesn't need to rotate through multiple models

## This is the line that is broken
- Either "signal_entity_index - 1" or "signal_entity_index + 1" is breaking it.  Presumably the +1 extends it too far

if (entities_df["pred_label"][signal_entity_index - 1] == "PER") and (entities_df["pred_label"][signal_entity_index + 1] == "LOC") and (entities_df["pred_start"][signal_entity_index + 1] - entities_df["pred_end"][signal_entity_index - 1] <= 20):
    689                 place = entities_df["pred_entity"][signal_entity_index + 1]
    690                 multiple = False

In [2]:
#no_test

#row names: people, places, events, json_path, entities, noCategory, validation_list
num_rows = 7
cols = np.zeros((num_rows,1))

path = 'models/75'
best_model_path = path + r"/" + "best_meta_model"

#"transcriptions\\15834.xml", "models/15834"
my_trans = "transcriptions\\15834.xml"
my_dir = best_model_path
people, places, events, json_path, entities, noCategory, validation_list = process_volume(my_trans, my_dir)
print("--------------------------------------")

Loaded model 'models/75/best_meta_model'
Entities extracted.
Enslaver found, but thinks principal is not enslaved... Principal dict:
[{'id': '15834-1033-3-P1;15834-1033-3-P3', 'name': 'Maria', 'origin': None, 'ethnicities': None, 'age': None, 'legitimacy': None, 'occupation': None, 'phenotype': None, 'status': 'Esc.a', 'titles': None, 'ranks': None, 'relationships': [{'related_person': '15834-1033-3-P6', 'relationship_type': 'godparent'}, {'related_person': '15834-1033-3-P5', 'relationship_type': 'enslaver'}]}]
-------------------------
Enslaver found, but thinks principal is not enslaved... Principal dict:
[{'id': '15834-1034-3-P1;15834-1034-3-P3', 'name': 'Antonia', 'origin': None, 'ethnicities': 'mina', 'age': 'adulta', 'legitimacy': None, 'occupation': None, 'phenotype': 'negra', 'status': 'Esc.va', 'titles': None, 'ranks': None, 'relationships': [{'related_person': '15834-1034-3-P5', 'relationship_type': 'godparent'}, {'related_person': '15834-1034-3-P4', 'relationship_type': 'ens

KeyError: 11

In [ ]:
#no_test
val_vis_bar(validation_list)

In [3]:
#no_test
from nbdev.export import notebook2script
notebook2script();

Converted 12-ssda-xml-parser.ipynb.
Converted 31-collate-xml-entities-spans.ipynb.
Converted 33-split-data.ipynb.
Converted 41-generic-framework-for-spacy-training.ipynb.
Converted 42-initial-model.ipynb.
Converted 51-data-preprocessing.ipynb.
Converted 52-unstructured-to-markup.ipynb.
Converted 53-markup-to-spatial-historian.ipynb.
Converted 54-utility-functions.ipynb.
Converted 61-prodigy-output-training-demo.ipynb.
Converted 62-full-model-application-demo.ipynb.
Converted 63-pt-model-training.ipynb.
Converted 64-es-model-training.ipynb.
Converted 65-all-annotations-model-training.ipynb.
Converted 66-es-guatemala-model-training.ipynb.
Converted 67-death-and-birth-records-together.ipynb.
Converted 70-exhaustive-training.ipynb.
Converted 71-relationship-builder.ipynb.
Converted 72-full-volume-processor.ipynb.
Converted 73-table-output.ipynb.
Converted 74-validation-visuals.ipynb.
Converted 75-meta-model.ipynb.
Converted 76-val-check-verif.ipynb.
